In [71]:
# !pip install -r ../exp_requirements.txt

In [72]:
# !pip uninstall virny -y

In [73]:
# Install using an HTTP link
# !pip install git+https://github.com/DataResponsibly/Virny.git@development

# Install using an SSH link
# !pip install git+ssh://git@github.com/DataResponsibly/Virny.git@development

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
import os
import warnings
warnings.filterwarnings('ignore')
os.environ["PYTHONWARNINGS"] = "ignore"

# Folktables GA Dataset With Random Nulls

## Import dependencies

In [3]:
import os
import copy

from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier

from virny.utils.custom_initializers import create_config_obj, create_models_metrics_dct_from_database_df
from virny.datasets.data_loaders import ACSEmploymentDataset
from source.user_interfaces.experiment_interface import run_exp_iter_with_models_stress_testing
from source.error_injectors.random_nulls_injector_v2 import RandomNullsInjectorV2
from source.utils.custom_initializers import create_experiment_data_loader
from source.utils.db_functions import read_model_metric_dfs_from_db
from source.preprocessing.basic_preprocessing import get_null_imputer_preprocessor

from configs.constants import NUM_METRICS_COMPUTATION_RUNS, EXPERIMENT_SEEDS, TEST_SET_FRACTION

## Initialize input variables for the experiment

In [4]:
ROOT_DIR = os.path.join(os.getcwd(), "..", "..")
EXPERIMENT_NAME = 'stress_testing_nulls'
DB_COLLECTION_NAME = f'{EXPERIMENT_NAME}_results'
SAVE_RESULTS_DIR_PATH = os.path.join(ROOT_DIR, 'results', EXPERIMENT_NAME)
COLUMNS_TO_TRANSFORM = ['SCHL', 'MAR', 'DIS', 'ESP', 'DEAR', 'DEYE', 'DREM', 'MIL']

In [5]:
data_loader = ACSEmploymentDataset(state=['GA'], year=2018, with_nulls=False, subsample_size=50_000, subsample_seed=42)
data_loader.X_data.head()

,MAR,MIL,ESP,MIG,DREM,NATIVITY,DIS,DEAR,DEYE,SEX,RAC1P,RELP,CIT,ANC,SCHL,AGEP
0,1,4,0,1,2,1,2,2,2,2,1,1,1,1,21,31
1,5,4,0,3,2,2,2,2,2,2,1,12,5,2,21,22
2,2,4,0,1,2,1,1,2,1,2,1,16,1,1,19,88
3,1,4,0,1,2,1,2,2,2,2,2,0,1,1,16,71
4,1,2,0,1,1,1,1,1,2,1,1,0,1,1,21,44


In [6]:
data_loader.full_df.isna().sum()

MAR         0
MIL         0
ESP         0
MIG         0
DREM        0
NATIVITY    0
DIS         0
DEAR        0
DEYE        0
SEX         0
RAC1P       0
RELP        0
CIT         0
ANC         0
SCHL        0
AGEP        0
dtype: int64

### Define a db writer and custom fields to insert into your database

In [7]:
from source.utils.db_functions import connect_to_mongodb

client, collection_obj, db_writer_func = connect_to_mongodb(DB_COLLECTION_NAME)

In [8]:
custom_table_fields_dct = {
    'error_type': 'Random Nulls',
    'error_percentages': '0%,5%,10%,20%,30%,40%,50%',
    'preprocessing_technique': 'cat: mode, num: median',
}

In [12]:
import uuid

# custom_table_fields_dct['session_uuid'] = 'c53d250b-5ba9-4d91-a444-ed7eb7919de5'
custom_table_fields_dct['session_uuid'] = str(uuid.uuid4())
print('Current session uuid: ', custom_table_fields_dct['session_uuid'])

Current session uuid:  c53d250b-5ba9-4d91-a444-ed7eb7919de5


### Create a metrics computation config object

In [83]:
config_yaml_path = 'experiment_config.yaml'
age_range = [i for i in range(30, 41)]
config_yaml_content = \
f"""
dataset_name: Folktables_GA_2018
bootstrap_fraction: 0.8
# n_estimators: 50
n_estimators: 10
# num_runs: {NUM_METRICS_COMPUTATION_RUNS}
num_runs: 2
sensitive_attributes_dct: {{'SEX': '1', 'RAC1P': '1', 'AGEP': {age_range}, 'SEX & RAC1P & AGEP': None}}
"""

with open(config_yaml_path, 'w', encoding='utf-8') as f:
    f.write(config_yaml_content)

In [84]:
metrics_computation_config = create_config_obj(config_yaml_path=config_yaml_path)

### Define tuning parameter for models

In [85]:
def get_models_params_for_tuning(models_tuning_seed):
    return {
        'DecisionTreeClassifier': {
            'model': DecisionTreeClassifier(random_state=models_tuning_seed),
            'params': {
                "max_depth": [20, 30],
                "min_samples_split" : [0.1],
                "max_features": ['sqrt'],
                "criterion": ["gini", "entropy"]
            }
        },
        'LogisticRegression': {
            'model': LogisticRegression(random_state=models_tuning_seed),
            'params': {
                'penalty': ['l2'],
                'C' : [0.0001, 0.1, 1, 100],
                'solver': ['newton-cg', 'lbfgs'],
                'max_iter': [250],
            }
        },
        # 'RandomForestClassifier': {
        #     'model': RandomForestClassifier(random_state=models_tuning_seed),
        #     'params': {
        #         "max_depth": [6, 10],
        #         "min_samples_leaf": [1],
        #         "n_estimators": [50, 100],
        #         "max_features": [0.6]
        #     }
        # },
        # 'XGBClassifier': {
        #     'model': XGBClassifier(random_state=models_tuning_seed, verbosity=0),
        #     'params': {
        #         'learning_rate': [0.1],
        #         'n_estimators': [200],
        #         'max_depth': [5, 7],
        #         'lambda':  [10, 100]
        #     }
        # },
        # 'KNeighborsClassifier': {
        #     'model': KNeighborsClassifier(),
        #     'params': {
        #         'n_neighbors' : [5, 7, 9, 11, 13, 15, 25],
        #         'weights' : ['uniform', 'distance'],
        #         'metric' : ['minkowski', 'euclidean', 'manhattan']
        #     }
        # },
        # 'MLPClassifier': {
        #     'model': MLPClassifier(random_state=models_tuning_seed),
        #     'params': {
        #         'hidden_layer_sizes':[(100,), (100,100,), (100,50,100,)],
        #         'activation': ['logistic', 'tanh', 'relu'],
        #         'solver': ['lbfgs', 'sgd', 'adam'],
        #         'learning_rate': ['constant', 'invscaling', 'adaptive']
        #     }
        # }
    }

## Run experiments

### Experiment iteration 1

In [86]:
# Configs for an experiment iteration
exp_iter_num = 1
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'
injector_config_lst = [0.05, 0.1, 0.2, 0.3, 0.4, 0.5]

error_injector = RandomNullsInjectorV2(experiment_seed, columns_to_transform=COLUMNS_TO_TRANSFORM, row_idx_nulls_percentage=0.0)
models_params_for_tuning = get_models_params_for_tuning(experiment_seed)
# Create a transformed data loader
exp_iter_data_loader = create_experiment_data_loader(data_loader)
preprocessor = get_null_imputer_preprocessor(exp_iter_data_loader)

In [87]:
run_exp_iter_with_models_stress_testing(data_loader=exp_iter_data_loader,
                                        experiment_seed=experiment_seed,
                                        test_set_fraction=TEST_SET_FRACTION,
                                        db_writer_func=db_writer_func,
                                        error_injector=error_injector,
                                        injector_config_lst=injector_config_lst,
                                        preprocessor=preprocessor,
                                        models_params_for_tuning=models_params_for_tuning,
                                        metrics_computation_config=metrics_computation_config,
                                        custom_table_fields_dct=custom_table_fields_dct,
                                        with_tuning=True,
                                        save_results_dir_path=SAVE_RESULTS_DIR_PATH,
                                        tuned_params_df_path=None,
                                        verbose=True)

2023-04-22 17:06:12 experiment_interface.py INFO    : Start an experiment iteration for the following custom params:


{'dataset_split_seed': 100,
 'error_percentages': '0%,5%,10%,20%,30%,40%,50%',
 'error_type': 'Random Nulls',
 'experiment_iteration': 'Exp_iter_1',
 'injector_config_lst': '[0.05, 0.1, 0.2, 0.3, 0.4, 0.5]',
 'model_init_seed': 100,
 'preprocessing_technique': 'cat: mode, num: median',
 'session_uuid': 'c53d250b-5ba9-4d91-a444-ed7eb7919de5'}




2023-04-22 17:06:12 experiment_interface.py INFO    : The dataset is preprocessed


Top indexes of an X_test in a base flow dataset:  Int64Index([ 4253, 30076, 48047,  1666, 30740, 13830, 34366,  5893, 45012,
            20187, 40178,  7958, 32251,  8493,  8752,  1984,  8898, 10170,
            20852, 19844],
           dtype='int64')
Top indexes of an y_test in a base flow dataset:  Int64Index([ 4253, 30076, 48047,  1666, 30740, 13830, 34366,  5893, 45012,
            20187, 40178,  7958, 32251,  8493,  8752,  1984,  8898, 10170,
            20852, 19844],
           dtype='int64')
2023/04/22, 17:06:13: Tuning DecisionTreeClassifier...
Fitting 3 folds for each of 4 candidates, totalling 12 fits
2023/04/22, 17:06:15: Tuning for DecisionTreeClassifier is finished [F1 score = 0.7493647181329992, Accuracy = 0.7602000000000001]

2023/04/22, 17:06:15: Tuning LogisticRegression...
Fitting 3 folds for each of 8 candidates, totalling 24 fits


2023-04-22 17:06:37 experiment_interface.py INFO    : Models are tuned and saved to a file


...

Classifiers testing by bootstrap:   0%|          | 0/10 [00:00<?, ?it/s]

[CV 2/3; 1/4] START criterion=gini, max_depth=20, max_features=sqrt, min_samples_split=0.1
[CV 2/3; 1/4] END criterion=gini, max_depth=20, max_features=sqrt, min_samples_split=0.1; Accuracy_Score: (test=0.764) F1_Score: (test=0.753) total time=   0.1s
[CV 3/3; 3/4] START criterion=entropy, max_depth=20, max_features=sqrt, min_samples_split=0.1
[CV 3/3; 3/4] END criterion=entropy, max_depth=20, max_features=sqrt, min_samples_split=0.1; Accuracy_Score: (test=0.751) F1_Score: (test=0.741) total time=   0.1s
[CV 2/3; 4/4] START criterion=entropy, max_depth=30, max_features=sqrt, min_samples_split=0.1
[CV 2/3; 4/4] END criterion=entropy, max_depth=30, max_features=sqrt, min_samples_split=0.1; Accuracy_Score: (test=0.761) F1_Score: (test=0.748) total time=   0.1s
[CV 1/3; 2/8] START C=0.0001, max_iter=250, penalty=l2, solver=lbfgs............
[CV 1/3; 2/8] END C=0.0001, max_iter=250, penalty=l2, solver=lbfgs; Accuracy_Score: (test=0.711) F1_Score: (test=0.669) total time=   0.3s
[CV 3/3; 4/8

Classifiers testing by bootstrap:   0%|          | 0/10 [00:00<?, ?it/s]

Classifiers testing by bootstrap:   0%|          | 0/10 [00:00<?, ?it/s]

Classifiers testing by bootstrap:   0%|          | 0/10 [00:00<?, ?it/s]

In [13]:
model_metric_dfs = read_model_metric_dfs_from_db(collection_obj, custom_table_fields_dct['session_uuid'])
models_metrics_dct = create_models_metrics_dct_from_database_df(model_metric_dfs)

In [18]:
models_metrics_dct[list(models_metrics_dct.keys())[0]].head(200)

,Metric,Bootstrap_Model_Seed,Model_Name,Model_Params,Run_Number,Dataset_Name,Num_Estimators,Test_Set_Index,Tag,Record_Create_Date_Time,...,Injector_Config_Lst,AGEP_dis,AGEP_priv,RAC1P_dis,RAC1P_priv,SEX&RAC1P&AGEP_dis,SEX&RAC1P&AGEP_priv,SEX_dis,SEX_priv,overall
0,Accuracy,101,DecisionTreeClassifier,"{'ccp_alpha': 0.0, 'class_weight': None, 'crit...",Run_1,Folktables_GA_2018,10,0,OK,2023-04-22 14:09:26.705,...,"[0.05, 0.1, 0.2, 0.3, 0.4, 0.5]",0.770941,0.819535,0.771368,0.782867,0.737063,0.905405,0.746923,0.813958,0.779100
1,Accuracy,101,DecisionTreeClassifier,"{'ccp_alpha': 0.0, 'class_weight': None, 'crit...",Run_1,Folktables_GA_2018,10,1,OK,2023-04-22 14:09:26.716,...,"[0.05, 0.1, 0.2, 0.3, 0.4, 0.5]",0.768537,0.815962,0.768010,0.780637,0.735664,0.899614,0.745962,0.809583,0.776500
2,Accuracy,101,DecisionTreeClassifier,"{'ccp_alpha': 0.0, 'class_weight': None, 'crit...",Run_1,Folktables_GA_2018,10,2,OK,2023-04-22 14:09:26.728,...,"[0.05, 0.1, 0.2, 0.3, 0.4, 0.5]",0.767456,0.813580,0.766484,0.779447,0.734965,0.893822,0.746154,0.806667,0.775200
3,Accuracy,101,DecisionTreeClassifier,"{'ccp_alpha': 0.0, 'class_weight': None, 'crit...",Run_1,Folktables_GA_2018,10,3,OK,2023-04-22 14:09:26.745,...,"[0.05, 0.1, 0.2, 0.3, 0.4, 0.5]",0.762889,0.805241,0.761600,0.774093,0.734266,0.886100,0.743269,0.798958,0.770000
4,Accuracy,101,DecisionTreeClassifier,"{'ccp_alpha': 0.0, 'class_weight': None, 'crit...",Run_1,Folktables_GA_2018,10,4,OK,2023-04-22 14:09:26.758,...,"[0.05, 0.1, 0.2, 0.3, 0.4, 0.5]",0.757481,0.794521,0.757021,0.766954,0.730769,0.874517,0.738654,0.790833,0.763700
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
384,Std,102,DecisionTreeClassifier,"{'ccp_alpha': 0.0, 'class_weight': None, 'crit...",Run_2,Folktables_GA_2018,10,6,OK,2023-04-22 14:12:06.054,...,"[0.05, 0.1, 0.2, 0.3, 0.4, 0.5]",0.108235,0.102747,0.107442,0.107251,0.095273,0.104002,0.097839,0.117578,0.107314
392,TNR,101,DecisionTreeClassifier,"{'ccp_alpha': 0.0, 'class_weight': None, 'crit...",Run_1,Folktables_GA_2018,10,0,OK,2023-04-22 14:09:26.705,...,"[0.05, 0.1, 0.2, 0.3, 0.4, 0.5]",0.698099,0.363874,0.638908,0.684048,0.628169,0.578947,0.613821,0.739583,0.668950
393,TNR,101,DecisionTreeClassifier,"{'ccp_alpha': 0.0, 'class_weight': None, 'crit...",Run_1,Folktables_GA_2018,10,1,OK,2023-04-22 14:09:26.716,...,"[0.05, 0.1, 0.2, 0.3, 0.4, 0.5]",0.699850,0.379581,0.643686,0.686106,0.632394,0.592105,0.616260,0.743229,0.671918
394,TNR,101,DecisionTreeClassifier,"{'ccp_alpha': 0.0, 'class_weight': None, 'crit...",Run_1,Folktables_GA_2018,10,2,OK,2023-04-22 14:09:26.728,...,"[0.05, 0.1, 0.2, 0.3, 0.4, 0.5]",0.702601,0.384817,0.645734,0.689537,0.633803,0.592105,0.620325,0.744792,0.674886


### Experiment iteration 2

In [90]:
client.close()